In [ ]:
!pip install tf-keras trl -q -q

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from trl import RewardTrainer, RewardConfig
import torch

MODEL_NAME = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=1)
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

dataset = load_dataset("trl-lib/ultrafeedback_binarized", split="train")

MAX_LEN = 256

def format_pair(example):
    chosen_text = example["chosen"][0]["content"]
    rejected_text = example["rejected"][0]["content"]

    chosen_tok = tokenizer(
        chosen_text,
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN,
    )
    rejected_tok = tokenizer(
        rejected_text,
        truncation=True,
        padding="max_length",
        max_length=MAX_LEN,
    )
    return {
        "chosen_input_ids": chosen_tok["input_ids"],
        "chosen_attention_mask": chosen_tok["attention_mask"],
        "rejected_input_ids": rejected_tok["input_ids"],
        "rejected_attention_mask": rejected_tok["attention_mask"],
    }

dataset = dataset.map(format_pair, remove_columns=dataset.column_names, batched=False)

dataset.set_format(type="torch")

sample = dataset[0]
assert "chosen_input_ids" in sample and "rejected_input_ids" in sample

config = RewardConfig(
    output_dir="./reward_model",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    logging_steps=50,
    fp16=False,
    bf16=False,
)

trainer = RewardTrainer(
    model=model,
    args=config,
    train_dataset=dataset,
)

trainer.train()


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/62135 [00:00<?, ? examples/s]

Filtering train >1024 tokens:   0%|          | 0/62135 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:wandb: You chose 'Create a W&B account'
wandb: Create an account here: https://wandb.ai/authorize?signup=true&ref=models
wandb: Paste an API key from your profile and hit enter:wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: damson4real94 (damson4real94-crescent-university-abeokuta) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
50,0.693100
100,0.693100
150,0.693100
200,0.693100
250,0.693100
300,0.693100
350,0.693100
400,0.693100
450,0.693100
